# Example WRIMS Search

This notebook is used to understand how WRIMS data can be combined with MarineRegion data and IJI genomics data to determine whether organisms detected in a sample are native/alien/invasive to the sample location. 

The input data is 
 - Observatory Information file that contains info on where the samples were taken,
 - Species Observation file that contains the animals and their AphiaID that was detected at each station


In [1]:
import pandas as pd
import invasive_checker.utils as utils
import requests

In [2]:
#Choose files to use for input and the name of the output file including WRIMS data
obs_info   = './data/ARMS_ObservatoryInfo.xlsx'
input_file = './data/ARMS_SpeciesPerObservatory_18S.xlsx'
output_file = './ARMS_SpeciesPerObservatory_18S_wrims.xlsx'

input_file = './data/ARMS_SpeciesPerObservatory_COI.xlsx'
output_file = './ARMS_SpeciesPerObservatory_COI_wrims.xlsx'

### Find the intersecting MRGID from MarineRegions using the new lat/lon tool
The ARMS_ObservatoryInfo Sheet file has the (averaged) lat, long for each Observatory station. This information is used to calculate the MarineRegion Geo ID's (MRGID) that intersect with the Observatory Lon/Lat. 

In [3]:
# Create dict of DataFrames; one dict per Excel sheet 
df_info_dict = pd.read_excel(obs_info,sheet_name = None)
df_info = df_info_dict['AveragedObservatoryInfo']
df_info.head()

,Observatory,Latitude_avg,Longitude_avg
0,Belgium.BE,51.51476,3.016295
1,Bodo.NO,67.24018,14.711630
2,Crete.GR,35.34489,25.207680
3,Redsea.IL,29.51278,34.927360
4,Gdynia.PL,54.48522,18.571110


In [11]:
mrgid_list = []
for index, row in df_info.iterrows():
    print(row['Observatory'])
    # print(row['Longitude_avg'], row['Latitude_avg'])
    # print('====')
    lat = row.Latitude_avg
    lon = row.Longitude_avg
    # Get info from https://www.marineregions.org/rest/getGazetteerRecordsByLatLong.json/{lat}/{lon}/'
    # for each lat/lon in the above file
    mrgids_sample_location = utils.get_mrgids(lon,lat)
    mrgids_sample_location['Observatory'] = row['Observatory']
    mrgids_sample_location['Latitude_avg'] = row['Latitude_avg']
    mrgids_sample_location['Longitude_avg'] = row['Longitude_avg']
    mrgid_list.append(mrgids_sample_location)
mrgid_df = pd.concat(mrgid_list)
mrgid_df.head()

Belgium.BE
Bodo.NO
Crete.GR
Redsea.IL
Gdynia.PL
Getxo.ES
Piran.SL
Koster.SE
Laso.DK
Limfjord.DK
Plymouth.UK
Roscoff.FR
Svalbard.NO
Tvarminne.FI
Vigo.ES


,MRGID,gazetteerSource,placeType,latitude,longitude,minLatitude,minLongitude,maxLatitude,maxLongitude,precision,preferredGazetteerName,preferredGazetteerNameLang,status,accepted,Observatory,Latitude_avg,Longitude_avg
0,49010,Flanders Marine Institute (2019). Maritime Bou...,Territorial Sea,51.327300,2.864670,51.08931,2.39405,51.55080,3.3704,NaN,Belgian 12 NM,English,standard,49010,Belgium.BE,51.51476,3.016295
1,3293,Flanders Marine Institute (2019). Maritime Bou...,EEZ,51.465450,2.705040,51.08931,2.23833,51.87611,3.3704,58302.49,Belgian Exclusive Economic Zone,English,standard,3293,Belgium.BE,51.51476,3.016295
2,3293,Flanders Marine Institute (2019). Maritime Bou...,EEZ,51.465450,2.705040,51.08931,2.23833,51.87611,3.3704,58302.49,Belgian Continental Shelf,English,standard,3293,Belgium.BE,51.51476,3.016295
3,26567,VLIZ (2020). Intersect of IHO Sea Areas and Ex...,Marine Region,51.465715,2.703969,NaN,NaN,NaN,NaN,NaN,Belgian part of the North Sea,English,synonym,3293,Belgium.BE,51.51476,3.016295
4,41272,International Council for the Exploration of t...,ICES Statistical Rectangles,51.750599,3.500000,51.50000,3.00000,52.00000,4.0000,NaN,32F3,English,standard,41272,Belgium.BE,51.51476,3.016295


### Join MRGID's

The input file is read and then, line-for-line, gets a ditribution status from WRIMS and then compares that to the MRGIDs from the Oservatory location. This is not using the old spatial checks (which ended up being very slow) but instead uses the new MR spatial API check


In [6]:
aphia_dict = {}
df_to_use = pd.read_excel(input_file,sheet_name = None)

for key, value in df_to_use.items():
    # For each sheet in the excel file:
    print(key)
    df_oi = df_to_use[key].copy() 
    df_oi = df_oi[df_oi['AphiaID_accepted'].notna()]
    df_oi['AphiaID_accepted'] =  df_oi['AphiaID_accepted'].astype(int)
    unique_aphias = df_oi['AphiaID_accepted'].unique()

    appended_data = []
    for aphiaID in unique_aphias:
        # For each unique aphia in the sample file
        xx, url = utils.get_aphia_status(aphiaID)
        if xx is None:
            pass
        else:
            xx['aphiaID'] = aphiaID
            appended_data.append(xx)
    wrims_df = pd.concat(appended_data)
    
    # Combine the MR info with the WRIMS info in order to find out what overlaps there are
    this_mrgid_df = mrgid_df[mrgid_df['Observatory'] == key]
    mrgid_merge = pd.merge(wrims_df, this_mrgid_df, how='inner', left_on='MRGID', right_on='MRGID') 
    mrgid_merge['establishmentMeans'] = mrgid_merge['establishmentMeans'].fillna(value='Present')
    mrgid_merge = mrgid_merge[['aphiaID','MRGID','locality','establishmentMeans']]
    
    # Combine the original file with the MRGID info for each AphiaID
    final_df = pd.merge(df_oi,mrgid_merge, how='left', left_on= 'AphiaID_accepted', right_on='aphiaID')
    final_df =final_df.drop(['aphiaID'], axis=1).drop_duplicates()
   
    aphia_dict[key] =  final_df 

Belgium.BE
No items found...
No items found...
No items found...
No items found...
No items found...
No items found...
No items found...
No items found...
No items found...
No items found...
No items found...
No items found...
No items found...
No items found...
No items found...
No items found...
No items found...
No items found...
No items found...
No items found...
No items found...
No items found...
No items found...
No items found...
No items found...
No items found...
No items found...
Going too fast!
No items found...
No items found...
No items found...
No items found...
No items found...
No items found...
No items found...
No items found...
No items found...
No items found...
Going too fast!
No items found...
No items found...
No items found...
No items found...
No items found...
No items found...
No items found...
No items found...
No items found...
No items found...
No items found...
No items found...
Going too fast!
No items found...
No items found...
No items found...
No it

### Convert back to excel

In [8]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter(output_file, engine='xlsxwriter')

for key, value in aphia_dict.items():
    # Write each dataframe to a different worksheet
    print(key)
    value.to_excel(writer, sheet_name=key)

# Close the Pandas Excel writer and output the Excel file.
writer.save()

Belgium.BE
Bodo.NO
Crete.GR
Redsea.IL
Gydnia.PL
Getxo.ES
Piran.SL
Koster.SE
Laso.DK
Limfjord.DK
Plymouth.UK
Roscoff.FR
Svalbard.NO
Tvarminne.FI
Vigo.ES
